pip install split-folders

In [1]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [4]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cuda")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])

    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/base-dataset-without-preprocessing/Without_any_preprocessing_small"
    output_folder = r"/kaggle/working/Images"

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    #splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Check for empty folders in the training and validation data
    train_path = os.path.join(output_folder, 'train')
    val_path = os.path.join(output_folder, 'val')

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=25, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/working/Images/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()

from sklearn.metrics import accuracy_score, precision_score, recall_score







# Define the ResNet model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, len(classes))
)
model = model.to(device)  # move the model to the specified device

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
START_LR = 0.0001
optimizer = optim.Adam(model.parameters(), lr=START_LR)
criterion = criterion.to(device)

# Train and Validate the model
for epoch in range(10):
    torch.cuda.empty_cache()
    print('Epoch {}/{}'.format(epoch + 1, 10))
    print('-' * 10)

    # Training phase
    model.train()
    running_loss = 0
    running_corrects = 0
    predictions = []
    true_labels = []
    all_labels = []
    all_predictions = []

    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        true_labels.extend(labels.cpu().numpy())

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(preds.cpu().numpy())
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

     # Calculate accuracy, precision, and recall for all classes
    acc = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)

    # Calculate accuracy for each class
    acc_per_class = {}
    for i, class_name in enumerate(classes):
        class_indices = [idx for idx, lbl in enumerate(true_labels) if lbl == i]
        class_true_labels = [true_labels[idx] for idx in class_indices]
        class_preds = [predictions[idx] for idx in class_indices]

        class_acc = accuracy_score(class_true_labels, class_preds)
        acc_per_class[class_name] = class_acc

    print('Accuracy for all classes: ', acc_per_class)
    print("")
        
    print('Precision for all classes: ', precision)
    print("")
    print('Recall for all classes: ', recall)
    print("")
    
    report_dict = classification_report(true_labels, predictions, target_names=classes, output_dict=True)
    report_pd = pd.DataFrame(report_dict)
    report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
    
    cnf_matrix = confusion_matrix(true_labels, predictions)
    df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index=classes, columns=classes)
    df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
    
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # Validation phase
    running_loss = 0
    running_corrects = 0
    predictions = []
    true_labels = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())

            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(val_dataset)
    epoch_acc = running_corrects.double() / len(val_dataset)

    report_dict = classification_report(true_labels, predictions, target_names=classes, output_dict=True)
    report_pd = pd.DataFrame(report_dict)
    report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')

    cnf_matrix = confusion_matrix(true_labels, predictions)
    df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index=classes, columns=classes)
    df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')

    print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    torch.cuda.empty_cache()

['grbcam1', 'chibat1', 'gobwea1', 'abhori1', 'reccuc1', 'slbgre1', 'somgre1', 'chespa1', 'yebsto1', 'brican1', 'afghor1', 'spwlap1', 'blakit1', 'combul2', 'pygbat1', 'whbtit5', 'grewoo2', 'whctur2', 'vilwea1', 'whbcan1', 'palfly2', 'nubwoo1', 'gobbun1', 'rebfir2', 'norpuf1', 'chtapa3', 'afrjac1', 'colsun2', 'carcha1', 'spewea1']


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 50.1MB/s]


Epoch 1/10
----------


100%|██████████| 71/71 [00:19<00:00,  3.56it/s]


Accuracy for all classes:  {'abhori1': 0.03, 'afghor1': 0.0, 'afrjac1': 0.0, 'blakit1': 0.5454545454545454, 'brican1': 0.0, 'carcha1': 0.11475409836065574, 'chespa1': 0.0, 'chibat1': 0.0, 'chtapa3': 0.0, 'colsun2': 0.2569444444444444, 'combul2': 0.5427350427350427, 'gobbun1': 0.0, 'gobwea1': 0.0, 'grbcam1': 0.0, 'grewoo2': 0.012195121951219513, 'norpuf1': 0.0, 'nubwoo1': 0.0, 'palfly2': 0.0, 'pygbat1': 0.09090909090909091, 'rebfir2': 0.0, 'reccuc1': 0.18556701030927836, 'slbgre1': 0.0, 'somgre1': 0.23270440251572327, 'spewea1': 0.0, 'spwlap1': 0.0, 'vilwea1': 0.0, 'whbcan1': 0.0, 'whbtit5': 0.0, 'whctur2': 0.0, 'yebsto1': 0.0}

Precision for all classes:  [0.16666667 0.         0.         0.2472885  0.         0.14583333
 0.         0.         0.         0.15744681 0.18594436 0.
 0.         0.         0.5        0.         0.         0.
 0.05882353 0.         0.3        0.         0.21022727 0.
 0.         0.         0.         0.         0.         0.        ]

Recall for all classes:

...

Val Loss: 1.1724 Acc: 0.6783
Epoch 10/10
----------


100%|██████████| 71/71 [00:14<00:00,  4.83it/s]


Accuracy for all classes:  {'abhori1': 0.94, 'afghor1': 0.8596491228070176, 'afrjac1': 0.6666666666666666, 'blakit1': 0.9808612440191388, 'brican1': 0.7391304347826086, 'carcha1': 0.9672131147540983, 'chespa1': 0.0, 'chibat1': 0.9365079365079365, 'chtapa3': 0.6190476190476191, 'colsun2': 0.9930555555555556, 'combul2': 0.9401709401709402, 'gobbun1': 0.96875, 'gobwea1': 0.0, 'grbcam1': 0.88, 'grewoo2': 0.9146341463414634, 'norpuf1': 0.25, 'nubwoo1': 0.29411764705882354, 'palfly2': 0.0, 'pygbat1': 0.36363636363636365, 'rebfir2': 0.7027027027027027, 'reccuc1': 0.9587628865979382, 'slbgre1': 0.05555555555555555, 'somgre1': 0.949685534591195, 'spewea1': 0.4, 'spwlap1': 0.9787234042553191, 'vilwea1': 0.9142857142857143, 'whbcan1': 0.0, 'whbtit5': 0.2222222222222222, 'whctur2': 0.0, 'yebsto1': 0.0}

Precision for all classes:  [0.93069307 0.875      0.57142857 0.98557692 0.56666667 0.93650794
 0.         0.67816092 0.59090909 0.9862069  0.93220339 0.87323944
 0.         0.89189189 0.91463415 0

100%|██████████| 19/19 [00:02<00:00,  6.70it/s]


Val Loss: 1.3739 Acc: 0.6849
